<a href="https://colab.research.google.com/github/GitaNadapdap/RepositoryRPL/blob/master/SMSSpamFilter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#connect to google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_file = '/content/drive/My Drive/Collab Notebook/SMSSPAMFILTER/portald1_data-collector.csv'

In [ ]:
import pandas as pd

data = pd.read_csv(data_file, header=None, names=['sms', 'nomor', 'label'])
data.head(500)

,sms,nomor,label
0,lni No Rek Nya Bank Mandiri 157-000-7197-883 A...,82321166619,Penipuan
1,Selamat no Anda trpilih Mndapatkan Hadiah ke-3...,85348849749,Penipuan
2,P3S4N R3SM1 SH0PEE Program tahunan 2020 no and...,85281914083,Penipuan
3,Pusing Dengan Masalah ekonomi Kami solusinya P...,85233034618,Penipuan
4,Anda Terpilih Pemenang cek Rp.175.000.000 PIN ...,85212204890,Penipuan
...,...,...,...
495,Pelanggan 081 27699 15441 +2000 COIN PULSA ...,85282859095,Penipuan
496,PESAN RESMI !! Anda pemenang SHOPEE AWAL TAH...,82358420264,Penipuan
497,"Pelanggan 082371 334121, Kamu Dapat +1000 COI...",85216045718,Penipuan
498,Ass bpk/ibu Selamat Anda resmi mendapatkan ...,82334331200,Penipuan


In [ ]:
#load stopwords and punctuation

import string
import nltk
import numpy as np
import re

nltk.download('stopwords')
nltk.download('punkt')

stopwords = nltk.corpus.stopwords.words('indonesian')
punctuation = string.punctuation

print(stopwords[:1000])
print(punctuation)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir', 'akhiri', 'akhirnya', 'aku', 'akulah', 'amat', 'amatlah', 'anda', 'andalah', 'antar', 'antara', 'antaranya', 'apa', 'apaan', 'apabila', 'apakah', 'apalagi', 'apatah', 'artinya', 'asal', 'asalkan', 'atas', 'atau', 'ataukah', 'ataupun', 'awal', 'awalnya', 'bagai', 'bagaikan', 'bagaimana', 'bagaimanakah', 'bagaimanapun', 'bagi', 'bagian', 'bahkan', 'bahwa', 'bahwasanya', 'baik', 'bakal', 'bakalan', 'balik', 'banyak', 'bapak', 'baru', 'bawah', 'beberapa', 'begini', 'beginian', 'beginikah', 'beginilah', 'begitu', 'begitukah', 'begitulah', 'begitupun', 'bekerja', 'belakang', 'belakangan', 'belum', 'belumlah', 'benar', 'benarkah', 'benarlah', 'berada', 'berakhir', 'berakhirla

In [ ]:
#pre-process sms content

def pre_process(sms):
  lowercase = "".join([char.lower() for char in sms if char not in punctuation])
  tokenize = nltk.tokenize.word_tokenize(lowercase)
  remove_stopwords = [word for word in tokenize if word not in stopwords]
  return remove_stopwords

data['processed'] = data['sms'].apply(lambda x: pre_process(x))
data.head(500)

,sms,nomor,label,processed
0,lni No Rek Nya Bank Mandiri 157-000-7197-883 A...,82321166619,Penipuan,"[lni, no, rek, nya, bank, mandiri, 15700071978..."
1,Selamat no Anda trpilih Mndapatkan Hadiah ke-3...,85348849749,Penipuan,"[selamat, no, trpilih, mndapatkan, hadiah, ke3..."
2,P3S4N R3SM1 SH0PEE Program tahunan 2020 no and...,85281914083,Penipuan,"[p3s4n, r3sm1, sh0pee, program, tahunan, 2020,..."
3,Pusing Dengan Masalah ekonomi Kami solusinya P...,85233034618,Penipuan,"[pusing, ekonomi, solusinya, pinjol, min, 5jt5..."
4,Anda Terpilih Pemenang cek Rp.175.000.000 PIN ...,85212204890,Penipuan,"[terpilih, pemenang, cek, rp175000000, pin, 25..."
...,...,...,...,...
495,Pelanggan 081 27699 15441 +2000 COIN PULSA ...,85282859095,Penipuan,"[pelanggan, 081, 27699, 15441, 2000, coin, pul..."
496,PESAN RESMI !! Anda pemenang SHOPEE AWAL TAH...,82358420264,Penipuan,"[pesan, resmi, pemenang, shopee, tah, unan, 12..."
497,"Pelanggan 082371 334121, Kamu Dapat +1000 COI...",85216045718,Penipuan,"[pelanggan, 082371, 334121, 1000, coin, pul, s..."
498,Ass bpk/ibu Selamat Anda resmi mendapatkan ...,82334331200,Penipuan,"[ass, bpkibu, selamat, resmi, hadiah, undian, ..."


In [ ]:
#categorizing ham/spam associated words

def categorize_words():
    Penipuan_words = []
    Iklanpromosi_words = []

    #spam associated words
    for sms in data['processed'][data['label'] == 'Penipuan']:
      for word in sms:
        Penipuan_words.append(word)

    for sms in data['processed'][data['label'] == 'Iklan/promosi']:
      for word in sms:
        Iklanpromosi_words.append(word)

    return Penipuan_words, Iklanpromosi_words


Penipuan_words, Iklanpromosi_words = categorize_words()

print(Penipuan_words[:5])
print(Iklanpromosi_words[:5])

['lni', 'no', 'rek', 'nya', 'bank']
['butuh', 'b14y4', 'tmbahan', 'mdal', 'us4h4']


In [ ]:
def predict(user_input):
  Penipuan_counter = 0
  Iklanpromosi_counter = 0 

  for word in user_input:
    Penipuan_counter += Penipuan_words.count(word)
    Iklanpromosi_counter += Iklanpromosi_words.count(word)

  print("********************************** RESULTS ***************************")
  if Iklanpromosi_counter > Penipuan_counter:
    accuracy = round((Iklanpromosi_counter / (Iklanpromosi_counter + Penipuan_counter)) * 100, 2)
    print("Message is not iklan/promosi, with {}% accuracy".format(accuracy))
  elif Penipuan_counter > Iklanpromosi_counter:
    accuracy = round((Penipuan_counter / (Iklanpromosi_counter + Penipuan_counter)) * 100, 2)
    print('Message is not Penipuan, with {}% accuray'.format(accuracy))
  else:
    print('Message could be Penipuan, with 50% accuracy')


In [ ]:
user_input = input('please type a Penipuan or Iklanpromosi message to check if our function predicts property\n')


please type a Penipuan or Iklanpromosi message to check if our function predicts property
no rek bank bca


In [ ]:
processed_input = pre_process(user_input)
predict(processed_input)

********************************** RESULTS ***************************
Message is not Penipuan, with 57.33% accuray
